In [3]:
#packages
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import matplotlib.pyplot as plt
import themepy
from highlight_text import HighlightText, ax_text, fig_text
import matplotlib.font_manager
from IPython.core.display import HTML
import time

In [ ]:
#fonts
def make_html(fontname):
    return "<p>{font}: <span style='font-family:{font}; font-size: 24px;'>{font}</p>".format(font=fontname)

code = "\n".join([make_html(font) for font in sorted(set([f.name for f in matplotlib.font_manager.fontManager.ttflist]))])

#HTML("<div style='column-count: 2;'>{}</div>".format(code))

In [4]:
#creating theme
theme = themepy.Theme()

theme.set_theme()

(theme
 .set_font("Open Sans", color="white") 
 .set_pips(False) 
 .set_spines("off", which=[], color="white") 
 .set_background("#001a4d") 
 .set_ticklabel_size(12) 
 .set_plot_colors("#DC5349","#26D8FF","#D3CFBD") 
)
theme.add_theme("pavel_dark")
theme.set_theme("pavel_dark")

Theme pavel_dark successfully overwritten


pavel_dark is the active theme

In [5]:
#function to scrape 2 seasons of data from fbref
def get_2_seasons_dfs(league_link):
    res = requests.get(league_link)
    soup = BeautifulSoup(res.content, 'html.parser')

    table=soup.find_all("table")[1]
    links=table.find_all("td",{"class":"left"})

    linksList=[]
    for i in range(0,len(links)):
        linksList.append("https://fbref.com"+(str(links[i]).split('href="',1)[1].split('">',1)[0]))

    linksList2=[]
    for i in range(0,len(linksList)):
        x=str(linksList[i]).split('/')
        x.append(x[-1])
        x[-2] = '2020-2021'
        x = '/'.join(x)
        linksList2.append(x)

    df_lists=[]
    for i in range(0, len(linksList)):
        df_lists.append(pd.read_html(linksList[i]))
        time.sleep(3)

    df_lists2=[]
    for i in range(0, len(linksList2)):
        df_lists2.append(pd.read_html(linksList2[i]))
        time.sleep(3)
    
    return (df_lists,df_lists2)

In [6]:
#scraping
df_lists,df_lists2=get_2_seasons_dfs("https://fbref.com/en/comps/9/2021-2022/2021-2022-Premier-League-Stats")

In [28]:
#function for timeline plot
def make_1season_timeline(df_lists,df_lists2,league, change,change_text,color,place,title,team,roll):
    fig, ax = plt.subplots(figsize=(16,9))
    plt.subplots_adjust(left=0.2, bottom=0.2, right=0.8, top=0.8)


    ax.grid(linewidth=.25, zorder=3, color="grey")

    for i in range(0, 18):   
        df=df_lists[i]
        temp=df[1]
        temp=temp[temp["Comp"]==league]
        df2=df_lists2[i]
        temp2=df2[1]
        temp2=temp2[temp2["Comp"]==league]
        temp2=temp2.iloc[-roll:]
        tmp=pd.concat([temp2,temp], axis=0)
        tmp.reset_index(inplace=True)
        tmp=tmp.drop(columns=["index","Match Report","Notes"])
        tmp = tmp.dropna(subset=["xG","xGA"])
        tmp.Date=pd.to_datetime(tmp.Date)
        tmp["xGD"]=tmp.xG-tmp.xGA
        tmp["xGD"]=pd.to_numeric(tmp["xGD"])
        tmp["xGD_MA5"]=tmp["xGD"].rolling(roll).mean()
        tmp = tmp.iloc[roll:]
        x=tmp.Date
        y=tmp.xGD_MA5
        ax.plot(x, y,c="grey",alpha=0.3,zorder=2)

    ax.axvline(x=pd.to_datetime(change), color='white',zorder=1,ls="--",lw=3,alpha=0.6)
    ax.text(x=pd.to_datetime(change_text), y=0, s="Managerial Change", 
        color = "w", alpha=0.4,verticalalignment='center',rotation=90,
        font={'size':12,'family':"Open Sans","weight":"medium"})

    df=df_lists[place]
    temp=df[1]
    temp=temp[temp["Comp"]==league]
    df2=df_lists2[place]
    temp2=df2[1]
    temp2=temp2[temp2["Comp"]==league]
    temp2=temp2.iloc[-roll:]
    tmp=pd.concat([temp2,temp], axis=0)
    tmp.reset_index(inplace=True)
    tmp=tmp.drop(columns=["index","Match Report","Notes"])
    tmp = tmp.dropna(subset=["xG","xGA"])
    tmp.Date=pd.to_datetime(tmp.Date)
    tmp["xGD"]=tmp.xG-tmp.xGA
    tmp["xGD"]=pd.to_numeric(tmp["xGD"])
    tmp["xGD_MA5"]=tmp["xGD"].rolling(roll).mean()
    tmp = tmp.iloc[roll:]
    x=tmp.Date
    y=tmp.xGD_MA5
    ax.plot(x, y,c=color,alpha=0.6,linewidth=3,zorder=4)



    ax.set_xticklabels([])
    ax.set_yticklabels([-3,-2,-1,0,1,2],alpha=0.6)
    ax.tick_params(labelright=True)

    fig_text(x = 0.5, y = 0.83,s="<{} ><against rest of the field>".format(team),ha='center', va='center',
    highlight_textprops=[{"color": color,'size':20,'family':"Open Sans","weight":"bold","alpha":0.6},
                {"color": "white",'size':20,'family':"Open Sans","weight":"bold","alpha":0.6}])
    fig_text(x = 0.5, y = 0.16,s="<{} - game moving average of expected goals difference>".format(roll),ha='center', va='center',
    highlight_textprops=[{"color": "white",'size':18,'family':"Open Sans","weight":"medium","alpha":0.4}])

    transform = ax.transAxes
    ax.text(0.5, -0.25, "Data: FBREF via Statsbomb\nInspired By: @markrstats, @DanielKatona17\nMade By: @pawellkaroll", 
                        transform = transform, 
                        ha = 'center',color = "w", alpha=0.4,
                        font={'size':12,"style":"italic"})
    ax.text(0.5, 1.25, str(league+", season 2021/2022"), 
                        transform = transform, 
                        ha = 'center',color = "w", alpha=0.4,
                        font={'size':14,'family':"Open Sans","weight":"medium"})
    ax.text(0.5, 1.10, title, 
                        transform = transform, 
                        ha = 'center',color = "w", alpha=0.8,
                        font={'size':30,'family':"Open Sans","weight":"bold"})

    #fig.savefig("D:\\Python_roboczy\\DataVisualization\\Timelines\\timeline_"+team+change+".png",dpi=400)

In [30]:
make_1season_timeline(df_lists=df_lists,df_lists2=df_lists2,league="Premier League", change="2021-11-08",change_text="2021-11-08",
color="#FDF105",place=10,title="How the things are going?",team="Newcastle",roll=3)

C:\Users\Paweł Zajdel\AppData\Local\Temp\ipykernel_5512\2396081590.py:58: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels([-3,-2,-1,0,1,2],alpha=0.6)
